# Week 11 - Statistical Data Analysis Using ML

이번 주에는 Machine Learning의 기초를 이해하고 Supervised Learning과 Unsupervised Learning을 통한 통계적 추론 방법을 학습한다.

## 1. Supervised Learning - 통계로 튀기는 치킨

통계청 수기공모전 최우수상 '통계로 튀기는 치킨'([링크](http://blog.naver.com/hi_nso/220489542903))

### 데이터 준비

* 먼저 각 feature(계절, 날씨, 이벤트 1~4)를 pandas DataFrame에 넣는다.
* 준비된 데이터는 8월 한 달치의 데이터이기 때문에 계절은 무시한다. 
* 날씨 및 각 이벤트는 categorical feature이므로 feature가 있으면 1, 없으면 0의 값을 갖도록 한다.

### 데이터 로드

In [ ]:
import pandas as pd
df = pd.read_csv('data/chicken.csv')

In [ ]:
# 데이터 확인
df

### Train/Test Set 분할

보통은 실제 데이터를 바탕으로 Train을 하고, 새로운 데이터를 기반으로 Test를 하지만, 본 연습에서는 주어진 데이터를 Train 데이터와 Test 데이터로 나누어 분석하고자 한다. 이를 위해 Scikit-Learn이 제공하는 train_test_split 모듈을 사용한다.

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2) # 80:20 으로 랜돔 분할

features = ['Weather', 'Weekend', 'Baseball', 'Temp']
train_X = df_train[features]
train_y = df_train['Sales']
test_X = df_test[features]
test_y = df_test['Sales']

In [ ]:
test_X

### Linear regression

#### Linear regression 수행

In [ ]:
from sklearn import linear_model

lm = linear_model.LinearRegression()
model = lm.fit(train_X, train_y)

#### 통계값 계산

In [ ]:
import numpy as np

# The coefficients
print('Coefficients: \n', lm.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((lm.predict(train_X) - train_y) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % lm.score(train_X, train_y))

#### (참고) statsmodels 모델 패키지 이용한 통계 계산

http://www.scipy-lectures.org/packages/statistics/index.html

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# 데이터 로드
# df = pd.read_csv('data/chicken.csv')

# 회귀분석 (독립~종속 데이터 구축/구분)
x = df[['Weather', 'Weekend', 'Baseball', 'Temp']]
y = df['Sales']

# 변수 x의 선형 예측 인자 작성
x = sm.add_constant(x)

# 모델 구축(분석)
est = sm.OLS(y, x).fit()

# 결과값 보기
est.summary()

#### 테스트 수행

In [ ]:
import numpy as np

pred = lm.predict(test_X)
sse = sum([x ** 2 for x in pred - test_y]) 
rmse = np.sqrt(sse / len(pred))
print('RMSE:', rmse)


* SSE: Sum of Square Error
* RMSE: Root Mean Square Error
    * 실험이나 관측에서 나타나는 오차(Error)를 제곱(Square)해서 평균(Mean)한 값의 제곱근(Root)
    * 값이 클 수록 정밀

#### 전체 판매량 재예측 및 시각화

학습된 lm 모델을 이용하여 전체 데이터를 다시 predict

In [ ]:
X = df[features]
pred = lm.predict(X)
df['Predict'] = pred
df

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

matplotlib.style.use('ggplot')

In [ ]:
df.plot(y=['Sales', 'Predict'], figsize=(15, 10))

## 2. Unsupervised Learning - 와인 성분 기반 클러스터링

### 데이터 로드

In [ ]:
import pandas as pd
df = pd.read_csv('data/wine.csv')

In [ ]:
# 데이터 확인
df

### 특성 정규화

K-Means clustering은 각 row vector의 Euclidean distance로 거리를 계산하기 때문에, 각 feature를 같은 unit으로 비교할 수 없다면 normalize를 해주는 것이 좋다.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(df)

In [ ]:
# 정규화된 데이터 체크
X[177]

### K-Means clustering 수행

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3)
kmeans.fit(X)

### K-Means clustering 결과 확인

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')

In [ ]:
colors = 'rgbyk'
colors = np.array([x for x in colors])
pred_cluster = kmeans.labels_.astype(np.int)
plt.scatter(X[:, 0], X[:, 2], color=colors[pred_cluster])
pred_cluster

### 데이터 프레임에 클러스터링 결과 추가

In [ ]:
df['cluster'] = pred_cluster
df

### PCA (Principle Component Analysis)

데이터의 컬럼이 11개이기 때문에 전체 데이터를 그래프에 나타낼 수 없다.

이럴 때 PCA(Principle Component Analysis)를 해주면 데이터가 가장 넓게 분포하는 축을 차례대로 찾아준다.

통계학에서 주성분 분석(主成分分析, Principal component analysis; PCA)이라고 부르는데 고차원의 데이터를 저차원의 데이터로 환원시켜준다.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X = pca.fit_transform(X)
plt.scatter(X[:, 0], X[:, 1], color=colors[pred_cluster])